In [8]:
import numpy as np
from math import floor
import torch.optim as optim
from matplotlib import pyplot as plt
from importlib import reload
import fcn # FCN is the module built from scratch. 
reload(fcn)

<module 'fcn' from '/home/glitchinthematrix/Desktop/EE-761/MAB-Hyperparameter-Tuning/fcn.py'>

In [4]:
#Preprocessing and Bifurcating  in Train and Test
"""
This block preprocesses images before they can be trained upon.
It does the followng:
- Scales the images betweeen 0 and 1
- Concatenates the HOG features to the flattened images
- Brings down the mean of the image to zero
- Dimensionality reduction using PCA. 3396 features to 1000 features
"""

data_raw = np.load('./dataset_2.npy')/255 #scaing images
labels = np.load('./dataset_2_label.npy')
hog_features = np.load('./dataset_2_hog.npy')[:,1:]
data= data_raw[1:,:]
labels = labels[:,1:]
y_train = labels[:,:9000]
y_test = labels[:,9000:]

X_train = data[:9000,:]
#X_trainm = X_train - np.mean(X_train, axis = 0) 
X_train = np.concatenate((X_train,hog_features[:,:9000].T),1) # Adding the HOG features in addition to the flattned attributes
X_trainm = X_train - np.mean(X_train, axis = 0) #Bringing mean down to 0
cov = np.dot(X_trainm.T, X_trainm) / X_trainm.shape[0] #Calculating covariance
U,S,V = np.linalg.svd(cov) #Calculating eigen vectors
X_train_uc = np.dot(X_trainm, U[:800].T) #Projecting to 1000 eigen vectors 

X_test = data[9000:,:]
#X_testm = X_test -  np.mean(X_train, axis = 0) 
X_test = np.concatenate((X_test,hog_features[:,9000:].T),1)
X_testm = X_test -  np.mean(X_train, axis = 0) 
X_test_uc = np.dot(X_testm, U[:800].T) 

#9000 train images and 1000 images in the test set


In [18]:
def run_config(config):
    layers = [800,config[0],config[1],2]
    discriminator = fcn.Model(layers,load=False,learning_rate=config[2]) #Initialize architecture
    discriminator.LoadData(X_train_uc,y_train.T,X_test_uc,y_test.T) #Load test and train data
    epoch_vec,train_vec,test_vec = discriminator.train(epoch=50)
    return test_vec[-1]/100

    

In [12]:
def query(lr_low, lr_high,hidden1_low, hidden1_high, hidden2_low, hidden2_high):
    
    arm_lr = lr_low + np.random.random()*(lr_high-lr_low)
    arm_nh1 = hidden1_low +floor(np.random.random()*(hidden1_high - hidden1_low))
    arm_nh2 = hidden2_low + floor(np.random.random()*(hidden2_high - hidden2_low))
    
    return [arm_nh1,arm_nh2,arm_lr]  

def binarization(prob):
    
    if np.random.random() < prob:
        return 1
    else:
        return 0

In [13]:

t = 0
Budget = 150
lr_low = 1e-4
lr_high = 1e-5
nh1_low = 500
nh1_high = 550
nh2_low = 50
nh2_high = 100

arms_query = [query(lr_low,lr_high,nh1_low,nh1_high,nh2_low,nh2_high)]
beta = 0.9

narms = 1
wins_arm = [0]
lose_arm = [0]

while t<Budget:
    arms_query.append(query(lr_low,lr_high,nh1_low,nh1_high,nh2_low,nh2_high))
    wins_arm.append(0)
    lose_arm.append(0)
    narms+=1
    
    reward_sample = [np.random.beta(wins_arm[i]+1,lose_arm[i]+1) for i in range(narms)]    
    I_1 = np.argmax(reward_sample)
    
    if np.random.random() > beta:
        
        reward_sample = [np.random.beta(wins_arm[i]+1,lose_arm[i]+1) for i in range(narms)]    
        I_1 = np.argmax(reward_sample)
    
    acc = run_config(arms_query[I_1])
    binary = binarization(acc)
    wins_arm[I_1] += binary
    lose_arm[I_1] += 1-binary

    t+=1
        
        


Linear added [800, 549]
Relu added
Linear added [549, 73]
Relu added
Linear added [73, 2]
Softmax added
epoch: 1, Discriminator Loss: 34.65920593380283
Train accuracy is: 53.422222222222224% and Validation accuracy is 53.0%
epoch: 2, Discriminator Loss: 34.65908032989729
Train accuracy is: 51.31111111111111% and Validation accuracy is 51.2%
epoch: 3, Discriminator Loss: 34.65889798421769
Train accuracy is: 50.855555555555554% and Validation accuracy is 50.8%
epoch: 4, Discriminator Loss: 34.658669953378634
Train accuracy is: 50.911111111111104% and Validation accuracy is 50.8%
epoch: 5, Discriminator Loss: 34.65839705662071
Train accuracy is: 50.98888888888888% and Validation accuracy is 51.0%
epoch: 6, Discriminator Loss: 34.658086434057026
Train accuracy is: 51.388888888888886% and Validation accuracy is 51.5%
epoch: 7, Discriminator Loss: 34.65772349761693
Train accuracy is: 52.0% and Validation accuracy is 52.1%
epoch: 8, Discriminator Loss: 34.65732043843398
Train accuracy is: 52.

Train accuracy is: 55.03333333333333% and Validation accuracy is 53.7%
epoch: 5, Discriminator Loss: 34.655630107658226
Train accuracy is: 58.84444444444444% and Validation accuracy is 57.199999999999996%
epoch: 6, Discriminator Loss: 34.65357993904763
Train accuracy is: 61.422222222222224% and Validation accuracy is 59.599999999999994%
epoch: 7, Discriminator Loss: 34.65130393123269
Train accuracy is: 63.21111111111111% and Validation accuracy is 61.8%
epoch: 8, Discriminator Loss: 34.648726409598865
Train accuracy is: 64.91111111111111% and Validation accuracy is 64.1%
epoch: 9, Discriminator Loss: 34.645778966943396
Train accuracy is: 66.38888888888889% and Validation accuracy is 65.9%
epoch: 10, Discriminator Loss: 34.64243766409449
Train accuracy is: 67.68888888888888% and Validation accuracy is 67.60000000000001%
epoch: 11, Discriminator Loss: 34.63862065861994
Train accuracy is: 68.5111111111111% and Validation accuracy is 69.0%
epoch: 12, Discriminator Loss: 34.63418583732529
T

Train accuracy is: 58.17777777777777% and Validation accuracy is 56.699999999999996%
epoch: 9, Discriminator Loss: 34.65370673935419
Train accuracy is: 59.05555555555555% and Validation accuracy is 57.9%
epoch: 10, Discriminator Loss: 34.65283589280257
Train accuracy is: 59.87777777777777% and Validation accuracy is 58.699999999999996%
epoch: 11, Discriminator Loss: 34.651907530144555
Train accuracy is: 60.87777777777777% and Validation accuracy is 59.599999999999994%
epoch: 12, Discriminator Loss: 34.650908483845576
Train accuracy is: 61.644444444444446% and Validation accuracy is 60.699999999999996%
epoch: 13, Discriminator Loss: 34.649841465629045
Train accuracy is: 62.522222222222226% and Validation accuracy is 61.9%
epoch: 14, Discriminator Loss: 34.648703730046336
Train accuracy is: 63.355555555555554% and Validation accuracy is 62.5%
epoch: 15, Discriminator Loss: 34.647493066598095
Train accuracy is: 63.98888888888889% and Validation accuracy is 63.5%
epoch: 16, Discriminator L

Train accuracy is: 69.8% and Validation accuracy is 70.6%
epoch: 14, Discriminator Loss: 34.59945504274435
Train accuracy is: 70.02222222222223% and Validation accuracy is 70.6%
epoch: 15, Discriminator Loss: 34.586017124399106
Train accuracy is: 70.37777777777777% and Validation accuracy is 70.8%
epoch: 16, Discriminator Loss: 34.56925972743279
Train accuracy is: 70.46666666666667% and Validation accuracy is 71.6%
epoch: 17, Discriminator Loss: 34.54809173762489
Train accuracy is: 70.77777777777777% and Validation accuracy is 71.2%
epoch: 18, Discriminator Loss: 34.52101134271715
Train accuracy is: 70.84444444444445% and Validation accuracy is 71.39999999999999%
epoch: 19, Discriminator Loss: 34.48585668892876
Train accuracy is: 70.95555555555556% and Validation accuracy is 71.5%
epoch: 20, Discriminator Loss: 34.43952145818116
Train accuracy is: 71.11111111111111% and Validation accuracy is 71.39999999999999%
Linear added [800, 501]
Relu added
Linear added [501, 91]
Relu added
Linear

Train accuracy is: 69.3% and Validation accuracy is 70.19999999999999%
epoch: 18, Discriminator Loss: 34.619520887991925
Train accuracy is: 69.62222222222222% and Validation accuracy is 70.39999999999999%
epoch: 19, Discriminator Loss: 34.61398530462216
Train accuracy is: 69.84444444444445% and Validation accuracy is 70.89999999999999%
epoch: 20, Discriminator Loss: 34.60764964642824
Train accuracy is: 70.12222222222222% and Validation accuracy is 70.8%
Linear added [800, 501]
Relu added
Linear added [501, 91]
Relu added
Linear added [91, 2]
Softmax added
epoch: 1, Discriminator Loss: 34.65809476771774
Train accuracy is: 55.111111111111114% and Validation accuracy is 52.6%
epoch: 2, Discriminator Loss: 34.65784035382214
Train accuracy is: 59.333333333333336% and Validation accuracy is 59.3%
epoch: 3, Discriminator Loss: 34.657564327105305
Train accuracy is: 61.74444444444445% and Validation accuracy is 62.3%
epoch: 4, Discriminator Loss: 34.657297738619874
Train accuracy is: 61.8333333

...

epoch: 9, Discriminator Loss: 34.65364825297193
Train accuracy is: 62.36666666666667% and Validation accuracy is 60.9%
epoch: 10, Discriminator Loss: 34.65196103015601
Train accuracy is: 63.98888888888889% and Validation accuracy is 62.1%
epoch: 11, Discriminator Loss: 34.6500453628918
Train accuracy is: 65.18888888888888% and Validation accuracy is 63.5%
epoch: 12, Discriminator Loss: 34.6478900320778
Train accuracy is: 66.35555555555555% and Validation accuracy is 65.3%
epoch: 13, Discriminator Loss: 34.64541785981965
Train accuracy is: 67.37777777777778% and Validation accuracy is 66.60000000000001%
epoch: 14, Discriminator Loss: 34.64257099691406
Train accuracy is: 68.12222222222222% and Validation accuracy is 68.10000000000001%
epoch: 15, Discriminator Loss: 34.639295509324185
Train accuracy is: 68.75555555555556% and Validation accuracy is 69.19999999999999%
epoch: 16, Discriminator Loss: 34.635474691938526
Train accuracy is: 69.35555555555555% and Validation accuracy is 69.5%
ep

Train accuracy is: 51.788888888888884% and Validation accuracy is 52.6%
epoch: 14, Discriminator Loss: 34.655540204015466
Train accuracy is: 51.94444444444445% and Validation accuracy is 52.5%
epoch: 15, Discriminator Loss: 34.655359596392145
Train accuracy is: 52.22222222222223% and Validation accuracy is 52.6%
epoch: 16, Discriminator Loss: 34.65517121804663
Train accuracy is: 52.5% and Validation accuracy is 52.6%
epoch: 17, Discriminator Loss: 34.654972576470286
Train accuracy is: 52.82222222222222% and Validation accuracy is 52.7%
epoch: 18, Discriminator Loss: 34.654767079301024
Train accuracy is: 53.233333333333334% and Validation accuracy is 52.800000000000004%
epoch: 19, Discriminator Loss: 34.654548676044385
Train accuracy is: 53.47777777777778% and Validation accuracy is 53.300000000000004%
epoch: 20, Discriminator Loss: 34.6543216781079
Train accuracy is: 53.800000000000004% and Validation accuracy is 53.5%
Linear added [800, 537]
Relu added
Linear added [537, 64]
Relu adde

epoch: 18, Discriminator Loss: 34.64494159630469
Train accuracy is: 64.7% and Validation accuracy is 63.5%
epoch: 19, Discriminator Loss: 34.643111402450735
Train accuracy is: 65.25555555555556% and Validation accuracy is 64.60000000000001%
epoch: 20, Discriminator Loss: 34.64112147888642
Train accuracy is: 66.07777777777778% and Validation accuracy is 65.4%
Linear added [800, 504]
Relu added
Linear added [504, 99]
Relu added
Linear added [99, 2]
Softmax added
epoch: 1, Discriminator Loss: 34.65960819006146
Train accuracy is: 49.08888888888889% and Validation accuracy is 48.5%
epoch: 2, Discriminator Loss: 34.659136743403955
Train accuracy is: 50.3% and Validation accuracy is 49.8%
epoch: 3, Discriminator Loss: 34.65874938004396
Train accuracy is: 50.3111111111111% and Validation accuracy is 49.9%
epoch: 4, Discriminator Loss: 34.65838806361799
Train accuracy is: 50.38888888888889% and Validation accuracy is 50.0%
epoch: 5, Discriminator Loss: 34.65800860829325
Train accuracy is: 50.6%

epoch: 2, Discriminator Loss: 34.66013913375369
Train accuracy is: 50.022222222222226% and Validation accuracy is 49.8%
epoch: 3, Discriminator Loss: 34.65939379493082
Train accuracy is: 50.044444444444444% and Validation accuracy is 49.8%
epoch: 4, Discriminator Loss: 34.65854796002659
Train accuracy is: 50.3111111111111% and Validation accuracy is 50.2%
epoch: 5, Discriminator Loss: 34.657580235107716
Train accuracy is: 52.077777777777776% and Validation accuracy is 51.6%
epoch: 6, Discriminator Loss: 34.65646681560482
Train accuracy is: 54.7% and Validation accuracy is 53.7%
epoch: 7, Discriminator Loss: 34.655201625953836
Train accuracy is: 57.06666666666666% and Validation accuracy is 56.10000000000001%
epoch: 8, Discriminator Loss: 34.65376523772253
Train accuracy is: 59.31111111111112% and Validation accuracy is 57.99999999999999%
epoch: 9, Discriminator Loss: 34.65212660078734
Train accuracy is: 60.988888888888894% and Validation accuracy is 59.5%
epoch: 10, Discriminator Loss:

epoch: 6, Discriminator Loss: 34.65635180218801
Train accuracy is: 57.91111111111111% and Validation accuracy is 57.4%
epoch: 7, Discriminator Loss: 34.65517793979129
Train accuracy is: 59.72222222222222% and Validation accuracy is 58.4%
epoch: 8, Discriminator Loss: 34.65390156954156
Train accuracy is: 61.31111111111112% and Validation accuracy is 59.199999999999996%
epoch: 9, Discriminator Loss: 34.65251812050367
Train accuracy is: 62.544444444444444% and Validation accuracy is 60.699999999999996%
epoch: 10, Discriminator Loss: 34.651034687159644
Train accuracy is: 63.72222222222222% and Validation accuracy is 62.3%
epoch: 11, Discriminator Loss: 34.649412765534514
Train accuracy is: 64.66666666666666% and Validation accuracy is 63.4%
epoch: 12, Discriminator Loss: 34.64762803059674
Train accuracy is: 65.52222222222223% and Validation accuracy is 64.8%
epoch: 13, Discriminator Loss: 34.645662940130606
Train accuracy is: 66.36666666666666% and Validation accuracy is 66.0%
epoch: 14, D

In [14]:
print(arms_query)

[[549, 73, 4.149078210212027e-05], [514, 98, 4.435349647893777e-05], [543, 66, 9.644220081145951e-05], [527, 77, 3.849717941567933e-05], [542, 99, 8.592549861068539e-05], [524, 66, 3.261483835782985e-05], [522, 66, 6.568930683043198e-05], [538, 63, 9.569143731085062e-05], [501, 91, 2.2449355360774296e-05], [538, 64, 8.265801610154727e-05], [506, 95, 6.403762051733852e-05], [530, 78, 8.662256706974362e-05], [544, 97, 1.4659945216529717e-05], [503, 59, 8.013742889316933e-05], [530, 73, 8.255774512224456e-05], [530, 99, 3.854524180251451e-05], [526, 93, 8.256343243268186e-05], [541, 66, 4.2821946750842236e-05], [511, 85, 4.666522096535015e-05], [502, 79, 1.2950041917506361e-05], [545, 52, 5.9411072677961215e-05], [510, 99, 3.5146254873056056e-05], [533, 72, 1.4964560002807853e-05], [514, 94, 7.970673736076801e-05], [510, 97, 3.8837184627546075e-05], [506, 97, 3.864009098684177e-05], [530, 94, 5.190492700991778e-05], [533, 79, 6.057520768808905e-05], [521, 85, 7.14035777724464e-05], [506, 

In [21]:
rewards_mean = [wins_arm[i]/(wins_arm[i]+lose_arm[i]+1e-4) for i in range(narms)]
print(np.argmax(rewards_mean))
print(rewards_mean)

41
[0.0, 0.49997500124993743, 0.874989062636717, 0.7499812504687383, 0.49997500124993743, 0.0, 0.0, 0.0, 0.8333194446759221, 0.49997500124993743, 0.0, 0.0, 0.0, 0.0, 0.7499812504687383, 0.0, 0.49997500124993743, 0.7999840003199936, 0.49997500124993743, 0.49997500124993743, 0.0, 0.6666444451851604, 0.49997500124993743, 0.7142755103498521, 0.49997500124993743, 0.6666444451851604, 0.0, 0.0, 0.9999000099990001, 0.49997500124993743, 0.9999000099990001, 0.7499812504687383, 0.0, 0.0, 0.9999000099990001, 0.49997500124993743, 0.49997500124993743, 0.49997500124993743, 0.0, 0.0, 0.0, 0.9999750006249845, 0.0, 0.0, 0.0, 0.0, 0.49997500124993743, 0.0, 0.0, 0.49997500124993743, 0.0, 0.9999000099990001, 0.49997500124993743, 0.49997500124993743, 0.9999666677777407, 0.0, 0.9999500024998749, 0.0, 0.49997500124993743, 0.49997500124993743, 0.9999000099990001, 0.0, 0.0, 0.0, 0.9999000099990001, 0.0, 0.9999000099990001, 0.0, 0.49997500124993743, 0.49997500124993743, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [17]:
print(arms_query[41])

[533, 56, 8.368783181086239e-05]


In [19]:
run_config(arms_query[41])


Linear added [800, 533]
Relu added
Linear added [533, 56]
Relu added
Linear added [56, 2]
Softmax added
epoch: 1, Discriminator Loss: 34.66070044962514
Train accuracy is: 61.977777777777774% and Validation accuracy is 62.2%
epoch: 2, Discriminator Loss: 34.65987293528827
Train accuracy is: 60.577777777777776% and Validation accuracy is 60.4%
epoch: 3, Discriminator Loss: 34.658916052659244
Train accuracy is: 60.65555555555555% and Validation accuracy is 59.599999999999994%
epoch: 4, Discriminator Loss: 34.657904175428996
Train accuracy is: 61.211111111111116% and Validation accuracy is 60.0%
epoch: 5, Discriminator Loss: 34.65680242384131
Train accuracy is: 62.11111111111111% and Validation accuracy is 60.5%
epoch: 6, Discriminator Loss: 34.65557453426991
Train accuracy is: 63.13333333333333% and Validation accuracy is 61.199999999999996%
epoch: 7, Discriminator Loss: 34.65417812702891
Train accuracy is: 64.12222222222222% and Validation accuracy is 62.4%
epoch: 8, Discriminator Loss: 

0.716